In [1]:
%%capture
pip install git+https://github.com/huggingface/speechbox

In [ ]:
%%capture
pip install pyannote.audio

In [ ]:
%%capture
pip install datasets

# Dataset

In [ ]:
from datasets import load_dataset

concatenated_librispeech = load_dataset(
    "sanchit-gandhi/concatenated_librispeech", split="train", streaming=True
)

# Creating the Pipelines

In [ ]:
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",
)

from pyannote.audio import Pipeline
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token="YOUR TOKEN")

from speechbox import ASRDiarizationPipeline
pipeline = ASRDiarizationPipeline(
    asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# We can now instantiate our combined diarization plus transcription pipeline, by passing the diarization model and ASR model to the ASRDiarizationPipeline class:
def tuple_to_string(start_end_tuple, ndigits=3):
    return str((round(start_end_tuple[0], ndigits), round(start_end_tuple[1], ndigits)))


def format_as_transcription(raw_segments):
    return [chunk for chunk in raw_segments]


# Separate file but idk how to separate it ;)

In [ ]:
# This is a 
import torch as pt
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer



class ComplexModel(nn.Module):
    def __init__(self, input_size=2*768, hidden_size=384):
        super(ComplexModel, self).__init__()
        
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Fully connected layer 1
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Fully connected layer 2
        self.fc3 = nn.Linear(hidden_size, hidden_size) # Fully connected layer 3
        self.fc4 = nn.Linear(hidden_size, hidden_size) # Fully connected layer 4
        self.fc5 = nn.Linear(hidden_size, 1)           # Output layer
        
        self.relu = nn.ReLU()     # ReLU activation function
        self.sigmoid = nn.Sigmoid() # Sigmoid activation function
        
        self.dropout = nn.Dropout(p=0.4) # Dropout layer to prevent overfitting

    def forward(self, x):
        # Forward pass
        x = self.fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        x = self.relu(x)
        
        x = self.fc4(x)
        x = self.relu(x)
        
        x = self.dropout(x)
        
        x = self.fc5(x)
        x = self.sigmoid(x)
        
        return x

In [ ]:
import torch as pt

In [ ]:
# def main():
device = pt.device("mps") if pt.backends.mps.is_available() else pt.device("cpu")
embedding_classifier = pt.load("is_conversation_embedding_classifier_doubles.pt")
embedding_classifier.to(device)
embedding_classifier.eval()
# Step 2: Create the embeddings
sent_transf = SentenceTransformer("avsolatorio/GIST-Embedding-v0", revision=None)
sent_transf.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

# Adapt Diarization Outputs to be inputed into model

A transcript is composed of turns. Each turn represents one participant's action in the conversation, which can include sending a message, asking a question, making a statement, etc. Turns are organized sequentially in the transcript to reflect the flow of the conversation. So, in essence, a transcript is a collection of turns exchanged between participants.

In [ ]:
class Turn:
    def __init__(self, turn) -> None:
        self.speaker = turn['speaker']
        self.text = turn['text']
        self.timestamp = turn['timestamp']
        self.embedding = sent_transf.encode(self.text)
    def __str__(self) -> str:
        return f"{self.speaker} {self.timestamp}: {self.text}"

In [ ]:
hola = Turn({'speaker': 'Carito', 'text': 'esto es una prueba', 'timestamp': (0.3, 0.4)})
print(hola)

Carito (0.3, 0.4): esto es una prueba


In [ ]:
class ComplexTranscript:
    def __init__(self, transcript):
        self.history: list[Turn] = []

        for t in transcript:
            new = Turn(t)
            self.history.append(new)


    def merge(self, transcript):
        for t in transcript:
            new = Turn(t)
            self.history.append(new)

        self.history = sorted(self.history, key=lambda x: x.timestamp[0])

    def fake_merge(self):
        timestamp = [
            (1, 3), 
            (3,6), 
            (6,9), 
            (9,12), 
            (12,15), 
            (15,18), 
            (18,21)
        ]
        text = [
            "What's the name of your dog?", 
            "Her name is Mila, she is a bernaddoodle", 
            "Oh that is so cute! My dog's name is Max.", 
            "I had a dog named Max once, but he passed away when I was a kid.", 
            "Really? That is so sad.",
            "Yeah, it was tough losing him, but now Mila brings so much joy into my life.",
            "I am so glad to hear that!"
        ]
        
        speaker = [
            "Speaker A",
            "Speaker B",
            "Speaker A",
            "Speaker B",
            "Speaker A",
            "Speaker B",
            "Speaker A",
        ]

        
        for i in range(7):
            fake_raw = {'speaker': speaker[i], 'text': text[i], 'timestamp':timestamp[i]}
            fake_turn = Turn(fake_raw)
            self.history.append(fake_turn)

        self.history = sorted(self.history, key=lambda x: x.timestamp[0])

    def len(self):
        return len(self.history)
    
    # def expire_speakers(self, conversation_members, curr_time):
    #     for speaker, expiry_time in conversation_members.items():
    #         if curr_time > expiry_time:
    #             conversation_members.pop(speaker)

In [ ]:
import librosa
# Load the MP3 file

def process_audio(filename1 = "Audios/Lady Yum.m4a", filename2 = "Audios/Lady Yum.m4a") -> ComplexTranscript: 
    audio1, _ = librosa.load(filename1, sr=16000)
    outputs1 = pipeline(audio1.copy())
    transcript1 = format_as_transcription(outputs1)
    
    the_transcript = ComplexTranscript(transcript1)

    # audio2, _ = librosa.load(filename1, sr=16000)
    # outputs2 = pipeline(audio2.copy())
    # transcript2 = format_as_transcription(outputs2)
    
    the_transcript.fake_merge()

    return the_transcript

    

In [ ]:
transcript = process_audio()


In [ ]:
class Conversation:
    def __init__(self, t: Turn) -> None:
        self.history: list[Turn] = [t]
        self.members: list[dict[str,float]]= {t.speaker:t.timestamp[1]}
        self.last: Turn = t

    def add(self, t: Turn) -> None:
        self.history.append(t)
        self.members[t.speaker] = t.timestamp[1]
        self.last = t
    
    def __str__(self) -> str:
        return str(self.members)

In [ ]:
def check_two_embeddings(e1, e2):
    e = pt.tensor(np.concatenate((e1, e2)))
    return embedding_classifier(e.to(device))[0].item()


Source for time tolerance
https://gtr.ukri.org/projects?ref=AH%2FF018908%2F1#:~:text=Participants%20in%20a%20conversation%20often,with%20that%20of%20another%20talker.

In [ ]:
conversations: list[Conversation] = []

time_tolerance = 0.13

for t in transcript.history:
    if len(conversations) == 0:
        conversations.append(Conversation(t))
        continue

    # Checking if the person is part of any conversation
    is_existing_member: bool = False
    for c in conversations:
        is_existing_member = is_existing_member or t.speaker in c.members

    # else compare against all other options:
    #dan los tiempos?
    potential_c = {}
    for idx, c in enumerate(conversations):
        if t.timestamp[0] >= (c.last.timestamp[0] + (c.last.timestamp[1] - c.last.timestamp[0]) * (1 - time_tolerance)):
            potential_c[idx] = check_two_embeddings(c.last.embedding, t.embedding)
            print("Checking two embeddings", c.last.text, "&&&&", t.text, potential_c[idx])
            # print(potential_c)
    
    if len(potential_c)>0:
        max_key = max(potential_c, key=lambda k: potential_c[k])
        if potential_c[max_key] < 0.5 and is_existing_member is False:
            print(max_key, t.text)
            conversations.append(Conversation(t))
        else:
            conversations[max_key].add(t)

    else:
        conversations.append(Conversation(t))


Checking two embeddings  Hello Antonio. &&&&  Hey! 0.9977649450302124
Checking two embeddings What's the name of your dog? &&&&  Hey! 0.9963232278823853
Checking two embeddings What's the name of your dog? &&&& Her name is Mila, she is a bernaddoodle 0.9710028171539307
Checking two embeddings  Hey! &&&&  How are you today? 0.902826189994812
Checking two embeddings Her name is Mila, she is a bernaddoodle &&&& Oh that is so cute! My dog's name is Max. 0.8712490200996399
Checking two embeddings  How are you today? &&&&  I'm good. You? 0.9897275567054749
Checking two embeddings  I'm good. You? &&&&  I'm great. What did you do today? 0.9887930750846863
Checking two embeddings Oh that is so cute! My dog's name is Max. &&&&  I'm great. What did you do today? 0.9691892266273499
Checking two embeddings Oh that is so cute! My dog's name is Max. &&&& I had a dog named Max once, but he passed away when I was a kid. 0.07672849297523499
Checking two embeddings  I'm great. What did you do today? &&&&

In [ ]:
for idx, c in enumerate(conversations):
    for l in c.history:
        print(idx, str(l))

0 SPEAKER_00 (0.0, 3.0):  Hello Antonio.
0 SPEAKER_01 (3.0, 5.0):  Hey!
0 SPEAKER_00 (5.0, 7.0):  How are you today?
0 SPEAKER_01 (7.0, 9.0):  I'm good. You?
0 SPEAKER_00 (9.0, 12.0):  I'm great. What did you do today?
0 SPEAKER_01 (12.0, 15.0):  I went for a long walk.
0 SPEAKER_00 (15.0, 17.0):  Oh, where did you go to?
0 SPEAKER_01 (17.0, 19.0):  To the pier.
0 SPEAKER_00 (19.0, 21.0):  Hmm, that's nice.
1 Speaker A (1, 3): What's the name of your dog?
1 Speaker B (3, 6): Her name is Mila, she is a bernaddoodle
1 Speaker A (6, 9): Oh that is so cute! My dog's name is Max.
1 Speaker B (9, 12): I had a dog named Max once, but he passed away when I was a kid.
1 Speaker A (12, 15): Really? That is so sad.
1 Speaker B (15, 18): Yeah, it was tough losing him, but now Mila brings so much joy into my life.
1 Speaker A (18, 21): I am so glad to hear that!


In [ ]:
# me encantaria que asi como se va actualizando la lista de conversation members, 
# tener algo que yo pueda constantemente estar revisando si la gente nueva esta o no esta en la conversacion